<a href="https://colab.research.google.com/github/olwynodpatterson/FYP/blob/main/10kForms_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Scraping 10k fillings

In [6]:
import requests
from bs4 import BeautifulSoup
import os
import time
import random

from google.colab import drive

# Function to retrieve a list of tickers from a given URL
def get_ticker_list(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    # Check if the request was successful
    if response.status_code == 200:
        tickers = response.text.splitlines()
        return tickers
    else:
        # Raise an exception if the request failed
        raise Exception(f"Failed to retrieve data. Status code: {response.status_code}")

# Function to choose a random CIK (Central Index Key) from the list of tickers
def choose_random_cik(tickers):
    # Choose a random ticker and associated CIK
    ticker, cik = random.choice(tickers).split('\t')
    return cik

# Function to get the URLs of 10-K filings for a given CIK
def get_10k_filing_urls(cik):
    time.sleep(3) # Delay to avoid overwhelming the server
    # Fetch the URLs for 10-K filings
    filings_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=10-K&dateb=&owner=exclude&count=10"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(filings_url, headers=headers)
    # Check response status
    if response.status_code != 200:
        print("Error fetching filings:", response.status_code)
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='tableFile2')
    if not table:
        return None

    # Parse the table to get the filing links
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            documents_page_link = 'https://www.sec.gov' + cols[1].a['href']
            submission_text_file_link = documents_page_link.replace('-index.htm', '.txt')
            return submission_text_file_link  # Return the first link found

    return None

def dehtml(html_content):
    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract text from the parsed HTML
    text = soup.get_text()

    return text

# Mount Google Drive
drive.mount('/content/drive')

def download_file(url, cik, folder='/content/drive/My Drive/10k_filings'):
    # Define the Google Drive folder path for the specific CIK
    drive_folder_path = os.path.join(folder, cik)

    # Create the folder if it doesn't exist
    if not os.path.exists(drive_folder_path):
        os.makedirs(drive_folder_path)

    # Correct the file paths to include the CIK-specific folder
    html_file_path = os.path.join(drive_folder_path, f'{cik}_10k_form.html')
    text_file_path = os.path.join(drive_folder_path, f'{cik}_10k_form.txt')

    # Make a request to download the file
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(html_file_path, 'wb') as html_file:
            for chunk in response.iter_content(chunk_size=8192):
                html_file.write(chunk)
        print(f"Downloaded HTML: {html_file_path}")

        # Read the HTML content
        with open(html_file_path, 'r', encoding='utf-8') as html_file:
            html_content = html_file.read()

        # Convert HTML to plain text
        plain_text = dehtml(html_content)

        # Save the plain text
        with open(text_file_path, 'w', encoding='utf-8') as text_file:
            text_file.write(plain_text)
        print(f"Converted to Text: {text_file_path}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")


# Main function to drive the program
def main():
    url = 'https://www.sec.gov/include/ticker.txt'
    tickers = get_ticker_list(url)
    i=0
    while i <1:
        cik =  choose_random_cik(tickers)
        print(f"Selected CIK: {cik}")
        filing_url = get_10k_filing_urls(cik)
        print(f"Filing URL: {filing_url}")
        if filing_url:
            download_file(filing_url, cik)
            time.sleep(1)  # Respectful delay between requests
            i += 1
        else:
            print(f"No 10-K filings found for CIK {cik}.")

if __name__ == "__main__":
    main()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Selected CIK: 1851174
Filing URL: https://www.sec.gov/Archives/edgar/data/1851174/000156459022012951/0001564590-22-012951.txt
Downloaded HTML: /content/drive/My Drive/10k_filings/1851174/1851174_10k_form.html
Converted to Text: /content/drive/My Drive/10k_filings/1851174/1851174_10k_form.txt


# Finding the Balance Sheet

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import os

# Directory containing your 10-K filings
base_directory_path = '/content/drive/My Drive/10k_filings'

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory_path):
    subdir_path = os.path.join(base_directory_path, subdir)

    # Check if the path is a directory to avoid trying to open files directly
    if os.path.isdir(subdir_path):
        # Loop through all files in the subdirectory
        for filename in os.listdir(subdir_path):
            # Check if the file is an HTML file
            if filename.endswith('.html'):
                print(filename)
                file_path = os.path.join(subdir_path, filename)

                # Open and read the file
                with open(file_path, 'r', encoding='utf-8') as file:
                    html_content = file.read()

                # Parse the HTML content
                soup = BeautifulSoup(html_content, 'html.parser')

                # Find the <a> tag that contains 'Consolidated Balance Sheet'
                a_tag = soup.find('a', string=lambda text: text and 'Balance Sheet' in text)
                #print(f"Found a tag {a_tag}")
                if a_tag and a_tag.has_attr('href'):
                    href = a_tag['href']

                    # Check if it's an internal link
                    if href.startswith('#'):
                        target_id = href[1:]  # Remove the '#' at the beginning
                        target_element = soup.find(id=target_id)

                        if target_element:
                            print(f"Found the target element in {filename}")

                            # Find the next table after the target element
                            next_table = target_element.find_next('table')
                            if next_table:
                                print(f"Found the next table in {filename}")
                                html_string = str(next_table)

                                # Use Pandas to read the HTML string; this returns a list of DataFrames
                                dfs = pd.read_html(html_string, header=1)

                                # Assuming there is only one table
                                table_df = dfs[0]

                                # Generate a dynamic CSV file path based on the HTML file name
                                csv_file_path = os.path.join(subdir_path, filename.replace('.html', '_balance_sheet.csv'))

                                # Export the DataFrame to a CSV file
                                table_df.to_csv(csv_file_path, index=False)
                                print(f"Table saved to {csv_file_path}")

                            else:
                                print(f"No next table found in {filename}.")

                        else:
                            print(f"Target element not found in {filename}.")
                    else:
                        print(f"The link in {filename} is not an internal link.")


1464423_10k_form.html
Found the target element in 1464423_10k_form.html
Found the next table in 1464423_10k_form.html
Table saved to /content/drive/My Drive/10k_filings/1464423/1464423_10k_form_balance_sheet.csv
1005229_10k_form.html
Found the target element in 1005229_10k_form.html
Found the next table in 1005229_10k_form.html
Table saved to /content/drive/My Drive/10k_filings/1005229/1005229_10k_form_balance_sheet.csv
1495932_10k_form.html
Found the target element in 1495932_10k_form.html
Found the next table in 1495932_10k_form.html
Table saved to /content/drive/My Drive/10k_filings/1495932/1495932_10k_form_balance_sheet.csv
1815753_10k_form.html
Found the target element in 1815753_10k_form.html
Found the next table in 1815753_10k_form.html
Table saved to /content/drive/My Drive/10k_filings/1815753/1815753_10k_form_balance_sheet.csv
1851174_10k_form.html
Found the target element in 1851174_10k_form.html
Found the next table in 1851174_10k_form.html
Table saved to /content/drive/My D

# Cleaning the Data

In [8]:
import pandas as pd
import os

def clean_columns_based_on_criteria(df):
    while len(df.columns) > 3:
        i = 0  # Start with the first column
        while i < len(df.columns) - 1:  # Ensure there is a next column to compare
            # Compare adjacent columns (ignoring the first row)
            if df.iloc[1:, i].equals(df.iloc[1:, i+1]):
                df.drop(df.columns[i], axis=1, inplace=True)  # Drop the first of the duplicates
            else:
                # Count nulls ignoring the first row
                nulls_first_col = df.iloc[1:, i].isnull().sum()
                nulls_second_col = df.iloc[1:, i+1].isnull().sum()
                # Drop the column with more nulls
                if nulls_first_col > nulls_second_col:
                    df.drop(df.columns[i], axis=1, inplace=True)
                    i -= 1  # Adjust index if a column is removed
                else:
                    df.drop(df.columns[i+1], axis=1, inplace=True)
            i += 1
    return df

def clean_csv_file(input_path, output_path):
    # Load the DataFrame from the CSV file
    df = pd.read_csv(input_path)

    # Apply the cleaning process
    cleaned_df = clean_columns_based_on_criteria(df)

    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_path, index=False)
    print(f"Cleaned file saved at {output_path}")

# Base directory containing your 10-K filings
base_input_directory = '/content/drive/My Drive/10k_filings'

# Iterate through all subdirectories in the base input directory
for subdir in os.listdir(base_input_directory):
    subdir_path = os.path.join(base_input_directory, subdir)
    if os.path.isdir(subdir_path):  # Ensure it's a directory
        # Process each CSV file within the subdirectory
        for filename in os.listdir(subdir_path):
            if filename.endswith('.csv'):
                input_path = os.path.join(subdir_path, filename)
                # Output file path remains within the same subdirectory
                output_path = os.path.join(subdir_path, filename.replace('.csv', '_cleaned.csv'))
                clean_csv_file(input_path, output_path)


Cleaned file saved at /content/drive/My Drive/10k_filings/1464423/1464423_10k_form_balance_sheet_cleaned.csv
Cleaned file saved at /content/drive/My Drive/10k_filings/1005229/1005229_10k_form_balance_sheet_cleaned.csv
Cleaned file saved at /content/drive/My Drive/10k_filings/1495932/1495932_10k_form_balance_sheet_cleaned.csv
Cleaned file saved at /content/drive/My Drive/10k_filings/1815753/1815753_10k_form_balance_sheet_cleaned.csv
Cleaned file saved at /content/drive/My Drive/10k_filings/1851174/1851174_10k_form_balance_sheet_cleaned.csv
